<img src="imgs/langchain.png" width="400">

**LangChain** es una biblioteca de Python diseñada para simplificar la creación de aplicaciones de lenguaje utilizando modelos de inteligencia artificial, como los basados en la arquitectura GPT de OpenAI. Su objetivo es proporcionar un marco de trabajo que permita a los desarrolladores integrar fácilmente capacidades avanzadas de procesamiento de lenguaje natural (PLN) en sus proyectos, especialmente en contextos donde se necesitan interacciones en lenguaje natural y se quiere explotar la potencia de los modelos de lenguaje grandes y complejos.

## **Clases principales**

Las clases principales de Langchain se pueden agrupar en las siguientes categorías:

1. **Prompts:**
   - `PromptTemplate`: Permite definir plantillas de prompts con variables.
   - `ChatPromptTemplate`: Permite definir plantillas de prompts para modelos de chat, como ChatGPT.
   - `FewShotPromptTemplate`: Permite definir plantillas de prompts para aprendizaje con pocos ejemplos (few-shot learning).

2. **Modelos de lenguaje (LLMs):**
   - `LLMChain`: Permite encadenar múltiples prompts y modelos de lenguaje.
   - `ChatOpenAI`: Interfaz para interactuar con el modelo de chat de OpenAI (ChatGPT).
   - `OpenAI`: Interfaz para interactuar con los modelos de lenguaje de OpenAI (GPT-3, Codex, etc.).

3. **Documentos y cargadores:**
   - `Document`: Representa un documento con texto y metadatos.
   - `TextLoader`: Carga documentos de texto desde archivos.
   - `PDFLoader`: Carga documentos PDF y extrae su texto.
   - `WebBaseLoader`: Carga documentos desde páginas web.

4. **Índices y almacenamiento:**
   - `VectorStore`: Almacena y recupera documentos utilizando embeddings vectoriales.
   - `FAISS`: Implementación de VectorStore basada en la biblioteca FAISS de Facebook.
   - `Chroma`: Implementación de VectorStore basada en la base de datos Chroma.

5. **Agentes y herramientas:**
   - `Tool`: Representa una herramienta externa que puede ser utilizada por un agente.
   - `Agent`: Define un agente que puede realizar tareas utilizando herramientas.
   - `AgentExecutor`: Ejecuta un agente y coordina la interacción con las herramientas.

6. **Cadenas y pipelines:**
   - `Chain`: Clase base para definir cadenas de componentes.
   - `SequentialChain`: Permite encadenar múltiples componentes en una secuencia.
   - `MapReduceChain`: Permite aplicar una cadena a múltiples entradas y combinar los resultados.
   - `TransformChain`: Permite aplicar una transformación a la salida de una cadena.

7. **Utilidades:**
   - `Memory`: Clase base para implementar memoria en las cadenas y agentes.
   - `ConversationBufferMemory`: Implementación de memoria que almacena el historial de conversación.
   - `ConversationSummaryMemory`: Implementación de memoria que resume el historial de conversación.


## **Ejemplos de uso**

Veamos algunos ejemplos de cómo se pueden utilizar las clases principales de Langchain para crear nuestras propias aplicaciones.

`langchain_openai`: Esta clase se utiliza para interactuar con los modelos de lenguaje de OpenAI a través de la API de OpenAI. Proporciona una interfaz para enviar solicitudes a los modelos de OpenAI, como GPT-3, y recibir las respuestas generadas. Con esta clase, puedes aprovechar la potencia de los modelos de lenguaje de OpenAI en tus aplicaciones.

In [2]:
# !pip install langchain-openai

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-turbo")

print(llm.invoke("¿Cuál es la capital de Francia?"))

content='La capital de Francia es París.' response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 17, 'total_tokens': 26}, 'model_name': 'gpt-4-turbo', 'system_fingerprint': 'fp_ea6eb70039', 'finish_reason': 'stop', 'logprobs': None} id='run-e11797e9-7fdc-4765-bca4-a970897f3cc9-0'


Si ahora queremos utilizar la API para conversaciones con el usuario, podemos hacerlo mediante las plantillas de *prompts*, incluidas en el módulo `langchain_core.prompts`.

`langchain_core`: Esta clase es el núcleo de Langchain y contiene las funcionalidades principales de la biblioteca. Proporciona las abstracciones y componentes básicos para construir cadenas de procesamiento de lenguaje natural. Incluye clases para representar documentos, consultas, agentes y otros elementos fundamentales en el flujo de trabajo de Langchain.

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un agente que contesta amablemente las preguntas de los usuarios."),
    ("user", "{input}")
])

chain = prompt | llm 

print(chain.invoke("¿Cuál es la capital de Francia?"))

content='La capital de Francia es París.' response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 39, 'total_tokens': 48}, 'model_name': 'gpt-4-turbo', 'system_fingerprint': 'fp_294de9593d', 'finish_reason': 'stop', 'logprobs': None} id='run-3a0a4652-a640-4981-afb6-a80f21c6b1c9-0'


Como vemos, obtenermos la respuesta que nos devuelve el modelo, que es una estructura de datos que contiene el texto generado por el modelo, así como información adicional como los tokens generados, nombre del modelo usado, etc. Para usar solo el texto utilizaremos el módulo `langchain_core.output_parsers`.

In [4]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

print(chain.invoke("¿Cuál es la capital de Francia?"))

La capital de Francia es París.


Vamos a ver cómo podemos crear un proceso que combine un contexto dado por un archivo de texto y una pregunta dada por el usuario.

In [5]:
from langchain.document_loaders import TextLoader
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

# Cargar el fichero de texto
loader = TextLoader('data_small.txt')
document = loader.load()

# Crear el prompt
prompt = ChatPromptTemplate.from_template("""Responde a las preguntas basándote solo en el siguiente contexto:
                                          
{context}

Question: {input}""")

chain = prompt | llm | output_parser

response = chain.invoke(
    {
        "context": document,
        "input": "¿Quiénes formaron el grupo Los Beatles?"
    }
)

print(response)

Los Beatles estuvo integrado por John Lennon, Paul McCartney, George Harrison y Ringo Starr.


¿Qué pasaría si nuestro fichero fuera muy grade y no cupiera en el contexto de nuestro modelo del lenguaje? Para esto casos dividiremos el documento en fragmentos de un cierto tamaño y los almacenaremos como documentos separados en una base de datos vectorial. A partir de la pregunta del usuario, buscaremos los fragmentos más relevantes y los pasaremos al modelo para obtener la respuesta.

In [13]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-turbo")

# Cargar el fichero de texto
loader = TextLoader('data.txt')
documents = loader.load()

# Dividir el texto en trozos
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

# Crear una instancia de OpenAIEmbeddings para generar los embeddings
embeddings = OpenAIEmbeddings()

# Almacenar los trozos de texto en Chroma
db = Chroma.from_documents(texts, embeddings)

Created a chunk of size 1364, which is longer than the specified 1000
Created a chunk of size 1608, which is longer than the specified 1000
Created a chunk of size 1269, which is longer than the specified 1000


Created a chunk of size 1067, which is longer than the specified 1000
Created a chunk of size 1091, which is longer than the specified 1000


In [14]:
document_chain = create_stuff_documents_chain(llm, prompt)

retriever = db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "¿Qué teorías hay sobre el origen del nombre de Los Beatles?"})

print("Contexto: ______________________________________________________")
for i, doc in enumerate(response["context"]):
    print(i+1, doc.page_content)
print("_________________________________________________________________")
    
print(response["answer"])

Contexto: ______________________________________________________
1 Décadas después surgieron dos versiones alternativas sobre el origen del nombre The Beatles. La primera proviene de George Harrison que sostuvo que fue tomada de la película The Wild One (El Salvaje), protagonizada por Marlon Brando, aunque la versión es discutible porque dicha película estuvo prohibida en Gran Bretaña hasta 1968. La segunda versión pertenece al poeta Royston Ellis, amigo de la banda y perteneciente al movimiento beat. Según Ellis, una noche de junio de 1960, se encontraba pasando el rato con Lennon y Sutcliffe, cuando surgió la cuestión del nuevo nombre de la banda. John entonces le respondió que se llamaría "The Beetles", deletreando la palabra con dos "E", significando Los Escarabajos". Ellis, que pertenecía al movimiento beat y se identificaba con los beatniks, propuso entonces el célebre cambio de la letra "E", por la letra "A". Ellis relató también que ese día él había cocinado un pollo que se pre

### **Recuperación de información de otras fuentes y con otros LLMs**

Vamos a utilizar un modelo de Antrhopic para recuperar información de un documento en formato PDF. 

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_anthropic import ChatAnthropic
from langchain_openai import OpenAIEmbeddings
import os

ANT_API_KEY = os.getenv("ANTRHOPIC_API_KEY")

llm = ChatAnthropic(model='claude-3-opus-20240229', api_key=ANT_API_KEY)

# Cargamos el documento PDF
loader = PyPDFLoader("https://sie.ulpgc.es/sites/default/files/Gu%C3%ADa%20ULPGC%202023%20(1).pdf")
documents = loader.load()

# Separamos el documento en trozos
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Almacenamos los trozos en Chroma
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(texts, embeddings)

# Crear un objeto RetrievalQA para responder preguntas
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff",
    retriever=db.as_retriever())


In [6]:
# Realizamos una pregunta y obtenemos la respuesta
query = "¿Cuántos estudiantes tiene la ULPGC?"
query = "¿Cuántos grupos de investigación hay en la ULPGC?"
query = "¿Cuántos créditos tienen los masters de la ULPGC?"
result = qa.invoke(query)

print(result)

{'query': '¿Cuántos créditos tienen los masters de la ULPGC?', 'result': 'Según la información proporcionada, los planes de estudios de los másteres universitarios de la ULPGC tendrán entre 60 y 120 créditos ECTS, equivalentes a uno o dos cursos académicos respectivamente.'}


Veamos cómo descargar un modelo de Hugging Face y utilizarlo para responder a una pregunta dada por el usuario.

In [7]:
import os
from langchain_community.llms import HuggingFaceEndpoint

HF_API_KEY = os.getenv("HF_API_KEY")

llm = HuggingFaceEndpoint(repo_id = "mistralai/Mistral-7B-Instruct-v0.2", huggingfacehub_api_token = HF_API_KEY)

print(llm("<s>[INST] ¿Cuál es la capital de Francia? [/INST]"))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/cayetano/.cache/huggingface/token
Login successful


/Users/cayetano/Propio/Notebooks/Machine Learning/RL/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 La capital de Francia es París. París es la ciudad más grande y más poblada de Francia, y es conocida por ser una de las ciudades más importantes del mundo en términos de arte, cultura y economía. Es un destino popular para viajeros de todo el mundo debido a sus famosas obras de arte, monumentos históricos, museos y otros lugares de interés.


### **Transcripciones**

In [ ]:
import yt_dlp

url = "https://www.youtube.com/watch?v=AHw2yOcK3-k"

ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

In [9]:
from openai import OpenAI
client = OpenAI()

audio_file= open("./Alsina.mp3", "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file
)
print(transcription.text)

¿Qué tal? ¿Cómo están? Bienvenidos a una nueva mañana de radio. Estamos el 6 de mayo de 2024, a seis días de San Pancracio. Seis días de que San Pancracio reparta suerte en las elecciones catalanas. Salvador Illa fue el primero en invocar el martirologio cristiano en este programa. Fue él quien nos recordó que el 12 de mayo es San Pancracio. Cuando digo que soy creyente, ayer me hicieron llegar una imagen de San Pancracio, porque el 10 de mayo vive San Pancracio. San Pancracio es un buen santo. Es un buen santo, sí, señor. El patrón de la suerte, aunque el propio Pancracio no tuviera mucha suerte, porque fue decapitado con 15 años, criatura. Pero la fortuna es verdad, sonría Salvador Illa, que cuenta en una entrevista que reza un padre nuestro cada día por la mañana. Y otra oración de creación propia que no dice cuál es. La fortuna sonríe a Salvador Illa porque llega a la última semana de campaña electoral en Cataluña liderando muy cómodamente las encuestas y habiendo mejorado la estim

In [12]:
# Save the transcription to a file
with open("transcription.txt", "w") as file:
    file.write(transcription.text)

### **Resúmenes**

¿Cómo se puede resumir un texto que es mayor que el contexto del LLM? Si el texto a resumir es mayor que el contexto máximo permitido por el modelo de lenguaje, puedes utilizar una técnica llamada "resumen recursivo" o "resumen jerárquico". Esta técnica consiste en dividir el texto en fragmentos más pequeños, resumir cada fragmento por separado y luego combinar los resúmenes en un resumen final. Hay que ajustar los valores de `chunk_size` y `chunk_overlap` según las necesidades y limitaciones del modelo de lenguaje que estés utilizando.

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain_openai import OpenAI

Fíjate que utilizamos la función `RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)` para dividir el texto en fragmentos de 1000 caracteres con un solapamiento de 50 caracteres. Luego, utilizamos el modelo de lenguaje para generar un resumen de cada fragmento y finalmente combinamos los resúmenes en un resumen final.

In [12]:
with open("transcription.txt") as f:
    texto = f.read()
    
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
textos = text_splitter.split_text(texto)

In [13]:
docs = [Document(page_content=t) for t in textos]

In [23]:
llm = OpenAI( 
    temperature=0
)
chain = load_summarize_chain(llm, chain_type="map_reduce") # max_tokens=1000

In [24]:
resumen = chain.invoke(docs)
print(resumen)

{'input_documents': [Document(page_content='¿Qué tal? ¿Cómo están? Bienvenidos a una nueva mañana de radio. Estamos el 6 de mayo de 2024, a seis días de San Pancracio. Seis días de que San Pancracio reparta suerte en las elecciones catalanas. Salvador Illa fue el primero en invocar el martirologio cristiano en este programa. Fue él quien nos recordó que el 12 de mayo es San Pancracio. Cuando digo que soy creyente, ayer me hicieron llegar una imagen de San Pancracio, porque el 10 de mayo vive San Pancracio. San Pancracio es un buen santo. Es un buen santo, sí, señor. El patrón de la suerte, aunque el propio Pancracio no tuviera mucha suerte, porque fue decapitado con 15 años, criatura. Pero la fortuna es verdad, sonría Salvador Illa, que cuenta en una entrevista que reza un padre nuestro cada día por la mañana. Y otra oración de creación propia que no dice cuál es. La fortuna sonríe a Salvador Illa porque llega a la última semana de campaña electoral en Cataluña liderando muy cómodament

Vemos que nos ha hecho el resumen en inglés. Si queremos que nos lo haga en español, podemos utilizar una plantilla que especifique el idioma.

In [26]:
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0)

prompt_template = """
Resuma el siguiente texto en español:

{text}

Resumen:
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=prompt_template,
)

chain = load_summarize_chain(llm, chain_type="map_reduce", return_intermediate_steps=True, map_prompt=prompt, combine_prompt=prompt)

text = docs[0]
summary = chain.invoke({"input_documents": [text]}, return_only_outputs=True)
print(summary["output_text"])


Hoy es 6 de mayo de 2024 y faltan seis días para las elecciones catalanas. Se dice que San Pancracio traerá suerte y Salvador Illa, quien es creyente, menciona que el 12 de mayo es su día. A pesar de haber sido decapitado a los 15 años, es considerado el patrón de la suerte. Illa reza cada mañana y tiene una oración para la fortuna. Actualmente, lidera las encuestas en la última semana de campaña en Cataluña.


Los parámetros `map_prompt` y `combine_prompt` en la función `load_summarize_chain` se utilizan para especificar los prompts que se utilizarán en las etapas de mapeo y combinación de la cadena de resumen, respectivamente.

`map_prompt`: Este parámetro se utiliza para especificar el prompt que se aplicará a cada documento individual durante la etapa de mapeo. En la etapa de mapeo, el modelo de lenguaje procesa cada documento por separado y genera un resumen intermedio para cada uno de ellos. Al establecer map_prompt=PROMPT, estás indicando que se utilice el prompt definido en la variable PROMPT para generar los resúmenes intermedios de cada documento.

`combine_prompt`: Este parámetro se utiliza para especificar el prompt que se aplicará durante la etapa de combinación. En la etapa de combinación, los resúmenes intermedios generados en la etapa de mapeo se combinan en un resumen final. Al establecer combine_prompt=PROMPT, estás indicando que se utilice el mismo prompt definido en la variable PROMPT para generar el resumen final a partir de los resúmenes intermedios.

In [ ]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
search.run("manchester united vs luton town match summary")

#### **Búsqueda de información en Google**
https://python.langchain.com/v0.1/docs/integrations/platforms/google

Veamos cómo podemos buscar información en Google y obtener respuestas a nuestras preguntas utilizando la API de Google Search.

In [16]:
import os

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

In [17]:
from langchain_google_community import GoogleSearchAPIWrapper
from langchain_core.tools import Tool

search = GoogleSearchAPIWrapper()

def top5_results(query):
    return search.results(query, num_results=5)

tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=top5_results,
)

In [18]:
res = tool.run("¿Qué es una calculadora?")

In [19]:
for r in res:
    print(r["title"])
    print(r["link"])
    print(r["snippet"])
    print()

Calculadora - Wikipedia, la enciclopedia libre
https://es.wikipedia.org/wiki/Calculadora
Una calculadora es un dispositivo que se utiliza para realizar cálculos aritméticos. Aunque las calculadoras modernas se incorporan a menudo en un ordenador ...

¿Qué es una calculadora y para qué sirve? - Definición
https://www.geeknetic.es/Calculadora/que-es-y-para-que-sirve
Dec 6, 2020 ... La calculadora científica sirve para realizar, prácticamente, cualquier tipo de cálculos, con ella podemos calcular superficies, sistemas de ...

Calculadora - Aplicaciones en Google Play
https://play.google.com/store/apps/details?id=com.google.android.calculator&hl=es
Con la Calculadora puedes hacer operaciones matemáticas sencillas y avanzadas en una aplicación con un diseño atractivo. • Haz operaciones básicas como ...

Calculadora de BMI
https://www.nhlbi.nih.gov/health/educational/lose_wt/BMI/bmi-m_sp.htm
El Indice de Masa Corporal (IMC) mide el contenido de grasa corporal en relación a la estatura y el p

In [20]:
# Importa beautifulsoup4 y requests
from bs4 import BeautifulSoup
import requests

# Extrae el contenido de la página
def extract_content(url):
    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.text, "html.parser")
    return soup.get_text()

# Preparamos todo el contenido de las páginas como una lista de documentos
def prepare_documents(results):
    documents = []
    for result in results:
        url = result["link"]
        content = extract_content(url)
        documents.append(content)
    return documents

# Extraemos el contenido de las páginas
documents = prepare_documents(res)

/Users/cayetano/Propio/Notebooks/Machine Learning/RL/env/lib/python3.10/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es.wikipedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/cayetano/Propio/Notebooks/Machine Learning/RL/env/lib/python3.10/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.geeknetic.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/cayetano/Propio/Notebooks/Machine Learning/RL/env/lib/python3.10/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'play.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.re

---

### Ejercicio

Prepara los documentos extraídos de la web para pasarlos como fuente de información al modelo de lenguaje y obterner respuestas a preguntas dadas por el usuario.

---


### **Agentes**

En LangChain, los agentes son un componente clave que permite crear sistemas capaces de realizar tareas complejas combinando múltiples herramientas y habilidades.

Algunas características importantes de los agentes en LangChain:

- Pueden acceder a una variedad de herramientas, como búsquedas web, calculadoras, bases de conocimiento, etc. Esto les permite obtener información y realizar acciones.

- Tienen una cadena de pensamiento que determina qué herramientas usar y en qué orden para completar una tarea dada. Esta cadena se construye usando prompts y modelos de lenguaje.

- Pueden iterar, es decir, usar la salida de una herramienta como entrada para la siguiente hasta lograr el objetivo.

- Permiten abstraer la complejidad, ya que el usuario sólo necesita proporcionar el objetivo de alto nivel y el agente determina los pasos necesarios.

- Son muy flexibles y extensibles. Se pueden agregar fácilmente nuevas herramientas y adaptar los agentes a diferentes dominios.

In [61]:
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
from langchain.tools import BaseTool

# Definir una herramienta simple
class SaludoTool(BaseTool):
    name = "Saludo"
    description = "Herramienta para saludar a alguien"

    def _run(self, nombre: str) -> str:
        return f"¡Hola {nombre}!"

    def _arun(self, nombre: str) -> str:
        raise NotImplementedError("SaludoTool no soporta ejecución asíncrona")

# Configurar el modelo de lenguaje (en este caso, OpenAI)
llm = OpenAI(temperature=0)

# Inicializar el agente con la herramienta y el modelo de lenguaje
tools = [SaludoTool()]
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# Ejecutar el agente con una entrada
entrada = "Saluda a Juan."
# entrada = "Saluda a Juan. También saludo a María."
# entrada = "Saluda a Juan. También saludo a María. A Pedro no hace falta que lo saludes."
resultado = agent.run(entrada)

print(resultado)



> Entering new AgentExecutor chain...
 I should greet Juan.
Action: Saludo
Action Input: Juan
Observation: ¡Hola Juan!
Thought: I should wait for Juan to respond.
Action: Saludo
Action Input: Juan
Observation: ¡Hola Juan!
Thought: I should listen to Juan's response.
Action: Saludo
Action Input: Juan
Observation: ¡Hola Juan!
Thought: I should respond to Juan's greeting.
Action: Saludo
Action Input: Juan
Observation: ¡Hola Juan!
Thought: I now know the final answer.
Final Answer: ¡Hola Juan!

> Finished chain.
¡Hola Juan!


In [62]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Saludo: Herramienta para saludar a alguien

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Saludo]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [91]:
from langchain.agents import initialize_agent, tool
from langchain.llms import OpenAI
from langchain.tools import Tool
from langchain.tools import DuckDuckGoSearchRun
from langchain_community.retrievers import WikipediaRetriever


# Configurar el modelo de lenguaje (en este caso, OpenAI)
llm = OpenAI(temperature=0)

# Definir las herramientas que el agente puede utilizar

#search = DuckDuckGoSearchRun()
wikipedia_retriever = WikipediaRetriever()

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

@tool
def search_wikipedia(query: str) -> str:
    """Searches Wikipedia for a query."""
    return wikipedia_retriever.invoke(query)

@tool
def current_time(input: str) -> str:
    """Returns the current time."""
    import datetime
    now = datetime.datetime.now()
    # return "11:30"
    return now.strftime("%H:%M")

@tool
def convert_number_to_binary(number: str) -> str:
    """Converts a number to binay."""
    return bin(int(number))[2:]


# Inicializar el agente con la herramienta y el modelo de lenguaje
tools = [
    # Tool(
    #     name="Búsqueda en Internet",
    #     func=search.run,
    #     description="Busca información en Internet utilizando DuckDuckGo"
    # ),
    # Tool(
    #     name="Longitud de una palabra",
    #     func=get_word_length,
    #     description="Devuelve la longitud de una palabra"
    # ),
    Tool(
        name="Búsqueda en Wikipedia",
        func=search_wikipedia,
        description="Busca información en Wikipedia"
    ),
    Tool(
        name="Hora actual",
        func=current_time,
        description="Devuelve la hora actual"
    ),
    Tool(
        name="Conversión de números a binario",
        func=convert_number_to_binary,
        description="Convierte un número a su representación binaria"
    )
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# Ejecutar el agente con una entrada
# entrada = "¿Cuántos caracteres tiene la palabra verborrea?"
# entrada = "¿Quién fue el primer presidente de Estados Unidos?"
# entrada = "¿Cuántos caracteres tiene el nombre completo de la ULPGC?"
# entrada = "¿Quién fue Benito Pérez Galdós?"
entrada = "La tienda está abierta desde las 10:00 hasta las 20:00 horas. ¿Está abierta ahora?"
# entrada = "Convierte a binario el año de nacimiento de Benito Pérez Galdós"

resultado = agent.run(entrada)

print(resultado)



> Entering new AgentExecutor chain...
 We need to check the current time and compare it to the store's opening and closing hours.
Action: Hora actual
Action Input: None
Observation: 14:14
Thought: We can see that it is currently 14:14, which falls within the store's opening and closing hours.
Action: Hora actual
Action Input: None
Observation: 14:14
Thought: We can also see that the store is open for 10 hours, from 10:00 to 20:00.
Action: Hora actual
Action Input: None
Observation: 14:14
Thought: Therefore, the store is currently open.
Final Answer: Yes, the store is currently open.

> Finished chain.
Yes, the store is currently open.


In [90]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Búsqueda en Wikipedia: Busca información en Wikipedia
Hora actual: Devuelve la hora actual
Conversión de números a binario: Convierte un número a su representación binaria

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Búsqueda en Wikipedia, Hora actual, Conversión de números a binario]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [49]:
from langchain_community.retrievers import WikipediaRetriever
etriever = WikipediaRetriever()
docs = retriever.invoke("Benito Pérez Galdós")
docs

In [ ]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="TheBloke/Mistral-7B-Instruct-v0.2-GGUF",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
  stream_options={"return_intermediate_steps": True}
)

for message in completion.choices[0].messages:
    print(message.role, message.content)



# print(completion.choices[0].message)